## 전체 소스 코드

In [18]:
import re
import uuid
import requests
from bs4 import BeautifulSoup
import rsa
import lzstring
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import os
import pandas as pd
import json
import math
from sqlalchemy import create_engine

class NaverCafe:
    
    def __init__(self):
        self.board_base_url_web = 'https://cafe.naver.com/autowave21/ArticleList.nhn'
        self.article_base_url_json = 'https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10295448/articles/'
        self.article_base_url_web = 'https://cafe.naver.com/autowave21/ArticleRead.nhn'
        self.comment_base_url = 'http://cafe.naver.com/CommentView.nhn'
        self.base_path = '디렉토리 경로'
        self.headers = { 
            'Content-Type': 'application/json; charset=utf-8',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        }
        
    ############################################################################################################
    ##### Naver 로그인
    ############################################################################################################
    
    def encrypt(self, key_str, uid, upw):
        def naver_style_join(l):
            return ''.join([chr(len(s)) + s for s in l])

        sessionkey, keyname, e_str, n_str = key_str.split(',')
        e, n = int(e_str, 16), int(n_str, 16)

        message = naver_style_join([sessionkey, uid, upw]).encode()

        pubkey = rsa.PublicKey(e, n)
        encrypted = rsa.encrypt(message, pubkey)

        return keyname, encrypted.hex()


    def encrypt_account(self, uid, upw):
        key_str = requests.get('https://nid.naver.com/login/ext/keys.nhn').content.decode("utf-8")
        return self.encrypt(key_str, uid, upw)


    def naver_session(self, nid, npw):
        encnm, encpw = self.encrypt_account(nid, npw)

        s = requests.Session()
        retries = Retry(
            total=5,
            backoff_factor=0.1,
            status_forcelist=[500, 502, 503, 504]
        )
        s.mount('https://', HTTPAdapter(max_retries=retries))
        request_headers = {
            'User-agent': 'Mozilla/5.0'
        }

        bvsd_uuid = uuid.uuid4()
        encData = '{"a":"%s-4","b":"1.3.4","d":[{"i":"id","b":{"a":["0,%s"]},"d":"%s","e":false,"f":false},{"i":"%s","e":true,"f":false}],"h":"1f","i":{"a":"Mozilla/5.0"}}' % (bvsd_uuid, nid, nid, npw)
        bvsd = '{"uuid":"%s","encData":"%s"}' % (bvsd_uuid, lzstring.LZString.compressToEncodedURIComponent(encData))

        resp = s.post('https://nid.naver.com/nidlogin.login', data={
            'svctype': '0',
            'enctp': '1',
            'encnm': encnm,
            'enc_url': 'http0X0.0000000000001P-10220.0000000.000000www.naver.com',
            'url': 'www.naver.com',
            'smart_level': '1',
            'encpw': encpw,
            'bvsd': bvsd
        }, headers=request_headers)

        finalize_url = re.search(r'location\.replace\("([^"]+)"\)', resp.content.decode("utf-8")).group(1)
        s.get(finalize_url)

        self.s = s
    
    ############################################################################################################
    ##### JSON 버전
    ############################################################################################################
    
    def getArticleHtmlJSON(self, article_id):
        url = self.article_base_url_json + str(article_id) + '?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665'
        html = self.s.get(url, headers = self.headers)
        df = pd.DataFrame([json.loads(html.text)['result']['article']])
        json_struct = json.loads(df.to_json(orient="records"))
        return pd.json_normalize(json_struct)
    
    def getCommentHtmlJSON(self, article_id):
        url = self.article_base_url_json + str(article_id) + '?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665'
        print(url)
        html = self.s.get(url, headers = self.headers)
        df = pd.DataFrame(json.loads(html.text)['result']['comments']['items'])
        df['article_id'] = article_id
        json_struct = json.loads(df.to_json(orient="records"))
        return pd.json_normalize(json_struct)
    
    ############################################################################################################
    ##### WEB 버전
    ############################################################################################################
    
    def getArticleListWEB(self, cafe_id, board_id, page):
        params = {
            'search.clubid' : cafe_id, # 카페 ID
            'search.menuid' : board_id, # 게시판 ID
            'search.boardtype' : 'L', # 보드타입 (반드시 필요로 하는 것은 아닌 것 같습니다.)
            'search.page' : page, # 불러올 페이지
            'userDisplay' : '50'
        }
        html = self.s.get(self.board_base_url, headers = self.headers, params = params)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
        board_name = self.getBoardName(board_id)
        values = []
        for tr in soup.select('tbody tr'):
            if len(tr.select('.inner_number')) > 0:
                values.append([ tr.select('.inner_number')[0].text.strip()
                              , tr.select('.article')[0].text.strip()
                              , board_name
                              , tr.select('.td_name a')[0].text.strip()
                              , tr.select('.td_date')[0].text.strip()
                              , 'https://cafe.naver.com/autowave21/'+tr.select('.inner_number')[0].text.strip() ])
        return pd.DataFrame(values, columns=['row_id', 'title_data', 'cat_data', 'user_data', 'create_date_data', 'link_data'])
    
    def getArticleHtmlWEB(self, cafe_id, board_id, article_id):
        params = {
            'clubid' : cafe_id, # 카페 ID
            'menuid' : board_id, # 게시판 ID
            'boardtype' : 'L', # 보드타입 (반드시 필요로 하는 것은 아닌 것 같습니다.)
            'articleid' : article_id  # 게시글 ID
        }   
        html = self.s.get(self.article_base_url_web, params = params, headers = self.headers)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
        link_data = 'https://cafe.naver.com/autowave21/' + str(article_id)
        board_name = self.getBoardName(board_id)
        if len(soup.select('.se-main-container')) > 0:
            title_data = soup.select('.fl .b')[0].text.strip()
            user_data = soup.select('.p-nick')[0].text.strip()
            create_date_data = soup.select('.fr .date')[0].text.strip()
            html_data = "".join(map(str, soup.select('.se-main-container')[0].contents))
        else :
            title_data = soup.select('.tit-box .fl .b')[0].text.strip()
            user_data = soup.select('.p-nick')[0].text.strip()
            create_date_data = soup.select('.tit-box .fr .date')[0].text.strip()
            html_data = "".join(map(str, soup.select('#tbody')[0].contents))            
        content_data = self.html2Text(board_id, article_id, html_data, '')
        data = [article_id, title_data, board_id, user_data, create_date_data, link_data, content_data, html_data]
        columns=['article_id', 'title_data', 'board_id', 'user_data', 'create_date_data', 'link_data', 'content_data', 'html_data']
        return pd.DataFrame([data], columns=columns)
        
    ############################################################################################################
    ##### 추가 작업 함수
    ############################################################################################################
    
    def makeDir(self, path):
        if not(os.path.isdir(path)):
            os.makedirs(os.path.join(path))
    
    async def downloadImage(self, url, path):
        loop = asyncio.get_event_loop()
        request = partial(requests.get, url, headers=self.headers)
        res = await loop.run_in_executor(None, request)
        with open(path, 'wb') as f:
            f.write(res.content)
            
    def getBoardName(self, board_id):
        if board_id == '655':
            return '정비인멤버 전용게시판'
        elif board_id == '334':
            return '노력멤버 전용게세판'
        elif board_id == '503':
            return '감사멤버 전용게시판'
        elif board_id == '3':
            return '공유멤버 전용게시판'
        elif board_id == '446':
            return '참여멤버 전용게시판'        

    def html2Text(self, board_name, row_id, text, elements):
        if text != None:
            if elements != None and len(elements) > 1:
                elements = eval(elements)
                keys = []
                values = []
                for i in range(len(elements)):
                    keys.append('[[[CONTENT-ELEMENT-'+str(i)+']]]')
                    if elements[i]['type'] == 'IMAGE':
                        values.append('<img src='+elements[i]['json']['image']['url']+' />')
                    else:
                        values.append('')
                for i in range(len(keys)):
                    text = text.replace(keys[i], values[i])
            pattern = r"<img.+?>"
            matchOB = re.findall(pattern , text, re.I|re.S)
            for i in range(len(matchOB)):
                try:
                    src = BeautifulSoup(matchOB[i], 'html.parser').select('img')[0].attrs['src'].split('?type')[0]
                except:
                    continue
                filename = str(row_id) +"_"+ str(i)+'.jpg'
                download_path = n.base_path+board_name+'\\'+str(row_id)
#                 n.makeDir(download_path)
#                 if not os.path.isfile(download_path+'\\'+filename):
#                     n.downloadImage(src, download_path+'\\'+filename)
                filename = '<p>\n<@@'+filename+'@@>\n</p>'
                text = text.replace(matchOB[i], filename)
            return BeautifulSoup(text, 'html.parser').text.strip()
        
    def makeComment(self, df):
        comment_json = dict()
        comment_num = 0
        comment_reply_num = 0
        comment_reply_all = dict()
        for i in range(len(df)) :
            if df['isRef'][i] == False:
                comment_num += 1
                comment_all = dict()
                comment_all["id"] = df['writer.id'][i]
                comment_all["name"] = df['writer.nick'][i]
                comment_all["text"] = df['content'][i]
                comment_all["reply"] = ''
                comment_json[str(comment_num)] = comment_all
                if comment_reply_num !=0 : 
                    comment_json[str(comment_num-1)]["reply"] = comment_reply_all
                    comment_reply_num = 0
                comment_reply_all = dict()
            else : 
                comment_reply_num += 1
                comment_reply = dict()
                comment_reply["id"] = df['writer.id'][i]
                comment_reply["name"] = df['writer.nick'][i]
                comment_reply["text"] = df['content'][i]
                comment_reply["reply"] = ''
                comment_reply_all[str(comment_num)+"_"+str(comment_reply_num)] = comment_reply
        comment_json[str(comment_num)]["reply"] = comment_reply_all
        comment_data = json.dumps(comment_json, indent = "\t", ensure_ascii=False)
        return comment_data

### 네이버 로그인
```python
    def encrypt(self, key_str, uid, upw):
        def naver_style_join(l):
            return ''.join([chr(len(s)) + s for s in l])

        sessionkey, keyname, e_str, n_str = key_str.split(',')
        e, n = int(e_str, 16), int(n_str, 16)

        message = naver_style_join([sessionkey, uid, upw]).encode()

        pubkey = rsa.PublicKey(e, n)
        encrypted = rsa.encrypt(message, pubkey)

        return keyname, encrypted.hex()


    def encrypt_account(self, uid, upw):
        key_str = requests.get('https://nid.naver.com/login/ext/keys.nhn').content.decode("utf-8")
        return self.encrypt(key_str, uid, upw)


    def naver_session(self, nid, npw):
        encnm, encpw = self.encrypt_account(nid, npw)

        s = requests.Session()
        retries = Retry(
            total=5,
            backoff_factor=0.1,
            status_forcelist=[500, 502, 503, 504]
        )
        s.mount('https://', HTTPAdapter(max_retries=retries))
        request_headers = {
            'User-agent': 'Mozilla/5.0'
        }

        bvsd_uuid = uuid.uuid4()
        encData = '{"a":"%s-4","b":"1.3.4","d":[{"i":"id","b":{"a":["0,%s"]},"d":"%s","e":false,"f":false},{"i":"%s","e":true,"f":false}],"h":"1f","i":{"a":"Mozilla/5.0"}}' % (bvsd_uuid, nid, nid, npw)
        bvsd = '{"uuid":"%s","encData":"%s"}' % (bvsd_uuid, lzstring.LZString.compressToEncodedURIComponent(encData))

        resp = s.post('https://nid.naver.com/nidlogin.login', data={
            'svctype': '0',
            'enctp': '1',
            'encnm': encnm,
            'enc_url': 'http0X0.0000000000001P-10220.0000000.000000www.naver.com',
            'url': 'www.naver.com',
            'smart_level': '1',
            'encpw': encpw,
            'bvsd': bvsd
        }, headers=request_headers)

        finalize_url = re.search(r'location\.replace\("([^"]+)"\)', resp.content.decode("utf-8")).group(1)
        s.get(finalize_url)

        self.s = s
```

In [19]:
n = NaverCafe()
n.naver_session('id', 'pw')

### 게시글 목록 수집
```python
    def getArticleListWEB(self, cafe_id, board_id, page):
        params = {
            'search.clubid' : cafe_id, # 카페 ID
            'search.menuid' : board_id, # 게시판 ID
            'search.boardtype' : 'L', # 보드타입 (반드시 필요로 하는 것은 아닌 것 같습니다.)
            'search.page' : page, # 불러올 페이지
            'userDisplay' : '50'
        }
        html = self.s.get(self.board_base_url, headers = self.headers, params = params)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
        board_name = self.getBoardName(board_id)
        values = []
        for tr in soup.select('tbody tr'):
            if len(tr.select('.inner_number')) > 0:
                values.append([ tr.select('.inner_number')[0].text.strip()
                              , tr.select('.article')[0].text.strip()
                              , board_name
                              , tr.select('.td_name a')[0].text.strip()
                              , tr.select('.td_date')[0].text.strip()
                              , 'https://cafe.naver.com/autowave21/'+tr.select('.inner_number')[0].text.strip() ])
        return pd.DataFrame(values, columns=['row_id', 'title_data', 'cat_data', 'user_data', 'create_date_data', 'link_data'])
```

In [6]:
n.getArticleListWEB(cafe_id='10295448', board_id='655', page='1')

https://cafe.naver.com/autowave21/ArticleList.nhn?search.clubid=10295448&search.menuid=655&search.boardtype=L&search.page=1&userDisplay=50


,row_id,title_data,cat_data,user_data,create_date_data,link_data
0,914395,액셀런 변천사 연장의 끝은 이 업 때려치면 케.,정비인멤버 전용게시판,영혼의 꿈,12:28,https://cafe.naver.com/autowave21/914395
1,914392,포터2오일쿨러 교환,정비인멤버 전용게시판,서연서진,12:18,https://cafe.naver.com/autowave21/914392
2,914391,포터 싸이드 불량,정비인멤버 전용게시판,고치고타,12:18,https://cafe.naver.com/autowave21/914391
3,914390,다들 아는 말통따기 전 이게편해요.,정비인멤버 전용게시판,영혼의 꿈,12:12,https://cafe.naver.com/autowave21/914390
4,914381,타***크 범죄영상이 올라왔네요,정비인멤버 전용게시판,카쎈,11:28,https://cafe.naver.com/autowave21/914381
5,914380,영업배상책임의 범위,정비인멤버 전용게시판,이제 마이믁었다,11:11,https://cafe.naver.com/autowave21/914380
6,914379,YF 히터코어 막힘...,정비인멤버 전용게시판,아이캔두잇,11:04,https://cafe.naver.com/autowave21/914379
7,914378,k7이나 hg lpg 레귤레이터 개선품,정비인멤버 전용게시판,아이캔두잇,11:03,https://cafe.naver.com/autowave21/914378
8,914372,포터2 브레이크 밀림으로,정비인멤버 전용게시판,경북카클리닉,10:27,https://cafe.naver.com/autowave21/914372
9,914366,#k5 오일 누유및 핸들유격,정비인멤버 전용게시판,향남행정카센타,10:01,https://cafe.naver.com/autowave21/914366


### 게시글 내용 수집
``` python
    def getArticleHtmlWEB(self, cafe_id, board_id, article_id):
        params = {
            'clubid' : cafe_id, # 카페 ID
            'menuid' : board_id, # 게시판 ID
            'boardtype' : 'L', # 보드타입 (반드시 필요로 하는 것은 아닌 것 같습니다.)
            'articleid' : article_id  # 게시글 ID
        }   
        html = self.s.get(self.article_base_url_web, params = params, headers = self.headers)
        print(html.url)
        soup = BeautifulSoup(html.text, 'html.parser')
        link_data = 'https://cafe.naver.com/autowave21/' + str(article_id)
        board_name = self.getBoardName(board_id)
        if len(soup.select('.se-main-container')) > 0:
            title_data = soup.select('.fl .b')[0].text.strip()
            user_data = soup.select('.p-nick')[0].text.strip()
            create_date_data = soup.select('.fr .date')[0].text.strip()
            html_data = "".join(map(str, soup.select('.se-main-container')[0].contents))
        else :
            title_data = soup.select('.tit-box .fl .b')[0].text.strip()
            user_data = soup.select('.p-nick')[0].text.strip()
            create_date_data = soup.select('.tit-box .fr .date')[0].text.strip()
            html_data = "".join(map(str, soup.select('#tbody')[0].contents))            
        content_data = self.html2Text(board_id, article_id, html_data, '')
        return [article_id, title_data, board_id, user_data, create_date_data, link_data, content_data, html_data] 

    def getArticleHtmlJSON(self, article_id):
        url = self.article_base_url_json + str(article_id) + '?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665'
        html = self.s.get(url, headers = self.headers)
        df = pd.DataFrame([json.loads(html.text)['result']['article']])
        json_struct = json.loads(df.to_json(orient="records"))
        return pd.json_normalize(json_struct)
```

- WEB 버전 수집

In [20]:
n.getArticleHtmlWEB(cafe_id='10295448', board_id='655', article_id='914338')

https://cafe.naver.com/autowave21/ArticleRead.nhn?clubid=10295448&menuid=655&boardtype=L&articleid=914338


,article_id,title_data,board_id,user_data,create_date_data,link_data,content_data,html_data
0,914338,말로만 전해듣던...,655,웅이군(lww1****),2020.10.20. 23:05,https://cafe.naver.com/autowave21/914338,시동이 안된다고 견인되서 점검하니 스타트모터가 작동은 안하고 배선은 이상없어 교체했...,"\n<div class=""se-component se-text se-l-defaul..."


- JSON 버전 수집

In [10]:
n.getArticleHtmlJSON(article_id='914338')

,id,refArticleId,subject,writeDate,readCount,commentCount,existScrapAddContent,contentHtml,customElements,gdid,...,writer.id,writer.nick,writer.image.url,writer.image.service,writer.image.type,writer.memberLevelName,writer.memberLevelIconUrl,decorator.isShowSuicideSaver,decorator.isPlug,template.isUse
0,914338,914338,말로만 전해듣던...,1603202703773,214,11,False,"<div class=""se-viewer se-theme-default"" lang=""...",[],90000004_009D1898000DF3A200000000,...,lww1004,웅이군,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,정비인멤버,https://cafe.pstatic.net/levelicon/1/1_140.gif,False,False,False


### 댓글 수집
``` python
    def getCommentHtmlJSON(self, article_id):
        url = self.article_base_url_json + str(article_id) + '?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665'
        print(url)
        html = self.s.get(url, headers = self.headers)
        df = pd.DataFrame(json.loads(html.text)['result']['comments']['items'])
        df['article_id'] = article_id
        json_struct = json.loads(df.to_json(orient="records"))
        return pd.json_normalize(json_struct)
```

In [21]:
n.getCommentHtmlJSON(article_id='914338')

https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10295448/articles/914338?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665


,id,refId,content,updateDate,isRef,isDeleted,isArticleWriter,isNew,isRemovable,sticker,...,writer.nick,writer.image.url,writer.image.service,writer.image.type,writer.image.isAnimated,sticker.id,sticker.url,sticker.type,sticker.width,sticker.height
0,67538477,67538477,물차 수고 하셨습니다,1603207757000,False,False,False,True,False,NaN,...,호기심 천국,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,NaN,NaN,NaN,NaN,NaN
1,67539415,67539415,차주분이 부동액을 엔진오일 주입구에 넣으셔서 일어났군요.\r\n수고하셨습니다,1603225786000,False,False,False,True,False,NaN,...,화성카센타티엠,https://cafethumb.pstatic.net/MjAxOTA4MTZfMjQy...,CAFE,c77_77,False,NaN,NaN,NaN,NaN,NaN
2,67540302,67540302,용감한 고객님~~ㅎㅎㅎ,1603237099000,False,False,False,True,False,NaN,...,참좋은모터스,https://cafethumb.pstatic.net/MjAxODExMjdfMjA2...,CAFE,c77_77,False,NaN,NaN,NaN,NaN,NaN
3,67540525,67540525,ㅎ ㅎ ㅎ 예전에 이스타나 본넷 얼고 가득 채우신분 생각 나네요 고생하셨습니다,1603238301000,False,False,False,True,False,NaN,...,전사장님,https://cafethumb.pstatic.net/MjAyMDA1MjFfNjYg...,CAFE,c77_77,False,NaN,NaN,NaN,NaN,NaN
4,67541535,67540525,전 처음봐서 한참 헤맸네요.. ㅎㅎㅎ 설마했는데..,1603243028000,True,False,True,True,False,NaN,...,웅이군,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,NaN,NaN,NaN,NaN,NaN
5,67540717,67540717,,1603239163000,False,False,False,True,False,NaN,...,혜선빠,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,ogq_58146d74c399f-11-185-160,https://storep-phinf.pstatic.net/ogq_58146d74c...,p100_100,129.0,112.0
6,67540935,67540935,수고 하셨습니다.,1603240148000,False,False,False,True,False,NaN,...,김기성,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,ogq_58146d74c399f-11-185-160,https://storep-phinf.pstatic.net/ogq_58146d74c...,p100_100,129.0,112.0
7,67541234,67541234,세상에 이런일이~~\r\n수고 많이 하셨습니다,1603241647000,False,False,False,True,False,NaN,...,혀기주니,https://blogpfthumb-phinf.pstatic.net/MjAxOTEw...,BLOG,c77_77,False,NaN,NaN,NaN,NaN,NaN
8,67541291,67541291,"엔진에 물넣으려면 과연 얼마를 넣어야 가득 넣을지, 배기,흡기 닫혀있을려나",1603241958000,False,False,False,True,False,NaN,...,드림자동차,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,NaN,NaN,NaN,NaN,NaN
9,67541529,67541291,코크로 배출한거만 약 20리터는 넘는거 같아요.\r\n실린더내부에도 들어가더라고요..,1603242995000,True,False,True,True,False,NaN,...,웅이군,https://ssl.pstatic.net/static/cafe/cafe_pc/de...,CAFE,c77_77,NaN,NaN,NaN,NaN,NaN,NaN


- 댓글 포맷 변경

In [22]:
n.makeComment(n.getCommentHtmlJSON(article_id='914338'))

https://apis.naver.com/cafe-web/cafe-articleapi/v2/cafes/10295448/articles/914338?page=1&useCafeId=true&buid=e4fe5ee7-8bbf-4d97-b875-c1d85e487665


'{\n\t"1": {\n\t\t"id": "csn3445",\n\t\t"name": "호기심 천국",\n\t\t"text": "물차 수고 하셨습니다",\n\t\t"reply": ""\n\t},\n\t"2": {\n\t\t"id": "attoo",\n\t\t"name": "화성카센타티엠",\n\t\t"text": "차주분이 부동액을 엔진오일 주입구에 넣으셔서 일어났군요.\\r\\n수고하셨습니다",\n\t\t"reply": ""\n\t},\n\t"3": {\n\t\t"id": "jjang_s_jin",\n\t\t"name": "참좋은모터스",\n\t\t"text": "용감한 고객님~~ㅎㅎㅎ",\n\t\t"reply": ""\n\t},\n\t"4": {\n\t\t"id": "lovelyokjjj",\n\t\t"name": "전사장님",\n\t\t"text": "ㅎ ㅎ ㅎ 예전에 이스타나 본넷 얼고 가득 채우신분 생각 나네요 고생하셨습니다",\n\t\t"reply": {\n\t\t\t"4_1": {\n\t\t\t\t"id": "lww1004",\n\t\t\t\t"name": "웅이군",\n\t\t\t\t"text": "전 처음봐서 한참 헤맸네요.. ㅎㅎㅎ 설마했는데..",\n\t\t\t\t"reply": ""\n\t\t\t}\n\t\t}\n\t},\n\t"5": {\n\t\t"id": "gu3207",\n\t\t"name": "혜선빠",\n\t\t"text": "",\n\t\t"reply": ""\n\t},\n\t"6": {\n\t\t"id": "daumpartner",\n\t\t"name": "김기성",\n\t\t"text": "수고 하셨습니다.",\n\t\t"reply": ""\n\t},\n\t"7": {\n\t\t"id": "han9878",\n\t\t"name": "혀기주니",\n\t\t"text": "세상에 이런일이~~\\r\\n수고 많이 하셨습니다",\n\t\t"reply": ""\n\t},\n\t"8": {\n\t\t"id": "qhalsdkqjwl",